# Estimation of tree height using GEDI dataset - Predictors extraction at point location

## 1.0 Reminder

You need to access the dataset we will use in this notebook and later too.
You already should have downloaded it and stored in the right folder, as follows:

    cd ~/SE_data
    git pull
    rsync -hvrPt --ignore-existing ~/SE_data/* /media/sf_LVM_shared/my_SE_data
    cd  /media/sf_LVM_shared/my_SE_data/exercise

    pip3 install gdown # Google Drive access

    [ -f tree_height.tar.gz ] || ~/.local/bin/gdown 1Y60EuLsfmTICTX-U_FxcE1odNAf04bd-
    [ -d tree_height ] || tar xvf tree_height.tar.gz
    
Also, you should already installed `rasterio` package as follows:
    
    pip3 install rasterio
    
*Hint: you should never use `pip3` under `sudo`, else you could break your system-wide Python environment in very creative and subtle ways.* 

## Introduction

The main objectives of this tutorial is to show several ways how to extract pixel value at point location. 

We will use the files stored at

In [27]:
! ls tree_height/txt/eu_x_y_*

tree_height/txt/eu_x_y_height_predictors_select.txt
tree_height/txt/eu_x_y_height_select.txt
tree_height/txt/eu_x_y_predictors_select.txt
tree_height/txt/eu_x_y_select.txt


In particular we will reproduce this data-table

In [2]:
! head -3 tree_height/txt/eu_x_y_predictors_select.txt # what we would get

ID X Y BLDFIE_WeigAver CECSOL_WeigAver CHELSA_bio18 CHELSA_bio4 convergence cti dev-magnitude eastness elev forestheight glad_ard_SVVI_max glad_ard_SVVI_med glad_ard_SVVI_min northness ORCDRC_WeigAver outlet_dist_dw_basin SBIO3_Isothermality_5_15cm SBIO4_Temperature_Seasonality_5_15cm treecover
1 6.050001 49.727499 1540 13 2113 5893 -10.4865598678589 -238043120 1.15841722488403 0.0690935924649239 353.983123779297 23 276.87109375 46.444091796875 347.665405273438 0.0424997806549072 9 780403 19.7989921569824 440.672210693359 85 
2 6.0500017 49.922155 1491 12 1993 5912 33.2743606567383 -208915344 -1.75534081459045 0.269112348556519 267.511688232422 19 -49.5263671875 19.552734375 -130.541748046875 0.182779803872108 16 772777 20.8894119262695 457.756195068359 85 


using the latitude longitude of each point

In [28]:
! head tree_height/txt/eu_x_y_select.txt

6.050001 49.727499
6.0500017 49.922155
6.0500021 48.602377
6.0500089 48.151979
6.0500102 49.58841
6.0500143 48.608456
6.0500165 48.571401
6.0500189 49.921613
6.0500201 48.822645
6.0500238 49.847522


In [1]:
! wc -l tree_height/txt/eu_x_y_select.txt # quite a lots of coords!

1267239 tree_height/txt/eu_x_y_select.txt


and the predictors 

In [30]:
! ls tree_height/geodata_raster/*.tif

tree_height/geodata_raster/BLDFIE_WeigAver.tif
tree_height/geodata_raster/CECSOL_WeigAver.tif
tree_height/geodata_raster/CHELSA_bio18.tif
tree_height/geodata_raster/CHELSA_bio4.tif
tree_height/geodata_raster/convergence.tif
tree_height/geodata_raster/cti.tif
tree_height/geodata_raster/dev-magnitude.tif
tree_height/geodata_raster/eastness.tif
tree_height/geodata_raster/elev.tif
tree_height/geodata_raster/forestheight.tif
tree_height/geodata_raster/glad_ard_SVVI_max.tif
tree_height/geodata_raster/glad_ard_SVVI_med.tif
tree_height/geodata_raster/glad_ard_SVVI_min.tif
tree_height/geodata_raster/latitude.tif
tree_height/geodata_raster/longitude.tif
tree_height/geodata_raster/northness.tif
tree_height/geodata_raster/ORCDRC_WeigAver.tif
tree_height/geodata_raster/outlet_dist_dw_basin.tif
tree_height/geodata_raster/SBIO3_Isothermality_5_15cm.tif
tree_height/geodata_raster/SBIO4_Temperature_Seasonality_5_15cm.tif
tree_height/geodata_raster/treecover.tif


## Running `rio` tool and using the `bash` shell and its tools
### by Francesco Lovergine

RasterIO has also a command line tool (`rio`) which can be used to perform a series of operation on rasters, which complementary in respect with the GDAL tools.If you install rasterio from distribution/kit it will reside in the ordinary system paths (note: you could find a `rasterio` binary instead of `rio`). If you install from the PyPI repository via `pip` it will install under `~/.local/bin`.

In [33]:
! rio --help

Usage: rio [OPTIONS] COMMAND [ARGS]...

  Rasterio command line interface.

Options:
  -v, --verbose           Increase verbosity.
  -q, --quiet             Decrease verbosity.
  --aws-profile TEXT      Select a profile from the AWS credentials file
  --aws-no-sign-requests  Make requests anonymously
  --aws-requester-pays    Requester pays data transfer costs
  --version               Show the version and exit.
  --gdal-version
  --show-versions         Show dependency versions
  --help                  Show this message and exit.

Commands:
  blocks     Write dataset blocks as GeoJSON features.
  bounds     Write bounding boxes to stdout as GeoJSON.
  calc       Raster data calculator.
  clip       Clip a raster to given bounds.
  convert    Copy and convert raster dataset.
  edit-info  Edit dataset metadata.
  env        Print information about the Rasterio environment.
  gcps       Print ground control points as GeoJSON.
  info       Print information about a data file.
  insp     

The goal of the next few snippets of code is creating in Python and rasterio. That can be done either by using th `rio` tool or completely by Python script.

First of all, the input for rio-sample submodule needs to be in list form, and that can be easily done via `sed` tool (or even `awk` if you prefer so):

In [3]:
! head tree_height/txt/eu_x_y_select.txt | sed -e 's/ /,/' -e 's/^/[/' -e 's/$/]/' # why and how?

[6.050001,49.727499]
[6.0500017,49.922155]
[6.0500021,48.602377]
[6.0500089,48.151979]
[6.0500102,49.58841]
[6.0500143,48.608456]
[6.0500165,48.571401]
[6.0500189,49.921613]
[6.0500201,48.822645]
[6.0500238,49.847522]


Once created a compatible input data file for `rio`, it can be used to sample every georaster. 

In [4]:
! head tree_height/txt/eu_x_y_select.txt | sed -e 's/ /,/' -e 's/^/[/' -e 's/$/]/' | rio sample tree_height/geodata_raster/BLDFIE_WeigAver.tif

[1540]
[1491]
[1521]
[1526]
[1547]
[1515]
[1520]
[1490]
[1554]
[1521]


Now, in principle you could stack together the input predictors and apply later the sampling. That could be done via `gdalbuildvrt` or `rio-stack`, BUT they both work currently only for homogeneous `dtype`, which is not our case, unfortunately.

*Homework: extract data types and sizes from all those files and check rasters are different types with the same dimensions. Hint: you can do that via rio or gdal tools or pktools* 

In [5]:
# rio stack `ls tree_height/geodata_raster/*.tif|grep -Ev 'latitude|longitude'` -o /tmp/tree_height_preds.tif
# gdalbuildvrt tree_height/geodata_raster/*.tif -o /tmp/predictors.vrt

One quite simple way to get the final result is using `rio-sample` to sample separately each field, then using `paste` to join together each per-field file.

In [24]:
%%bash

# this is VERY slow on 1M of records...
time (
FILES=$(awk '{if (NR==1) print $4,$5,$6,$7,$8,$9,$10,$11,$12,$13,$14,$15,$16,$17,$18,$19,$20,$21,$22}' tree_height/txt/eu_x_y_predictors_select.txt)
for n in $FILES 
do 
    sed -e 's/ /,/' -e 's/^/[/' -e 's/$/]/' tree_height/txt/eu_x_y_select.txt | rio sample tree_height/geodata_raster/$n.tif | tr -d '[]' > /tmp/$n
done

seq $(wc -l tree_height/txt/eu_x_y_select.txt|cut -d' ' -f1) >/tmp/ids
echo 'ID X Y' "$FILES" >  tree_height/txt/eu_x_y_predictors_select_rio.txt
paste -d ' ' /tmp/ids tree_height/txt/eu_x_y_select.txt \
$(for n in  $(awk '{if (NR==1) print $4,$5,$6,$7,$8,$9,$10,$11,$12,$13,$14,$15,$16,$17,$18,$19,$20,$21,$22}' tree_height/txt/eu_x_y_predictors_select.txt); do ls /tmp/$n ; done) \
>> tree_height/txt/eu_x_y_predictors_select_rio.txt 
)



real	20m51.133s
user	18m36.829s
sys	2m39.808s


Running time

    real	20m51.133s
    user	18m36.829s
    sys 	2m39.808s


In [25]:
! head -3  tree_height/txt/eu_x_y_predictors_select_rio.txt 

ID X Y BLDFIE_WeigAver CECSOL_WeigAver CHELSA_bio18 CHELSA_bio4 convergence cti dev-magnitude eastness elev forestheight glad_ard_SVVI_max glad_ard_SVVI_med glad_ard_SVVI_min northness ORCDRC_WeigAver outlet_dist_dw_basin SBIO3_Isothermality_5_15cm SBIO4_Temperature_Seasonality_5_15cm treecover
1 6.050001 49.727499 1540 13 2113 5893 -10.486559867858887 -238043120 1.1584172248840332 0.06909359246492386 353.9831237792969 23 276.87109375 46.444091796875 347.6654052734375 0.04249978065490723 9 780403 19.798992156982422 440.6722106933594 85
2 6.0500017 49.922155 1491 12 1993 5912 33.27436065673828 -208915344 -1.755340814590454 0.26911234855651855 267.5116882324219 19 -49.5263671875 19.552734375 -130.541748046875 0.18277980387210846 16 772777 20.88941192626953 457.7561950683594 85


## Running `gdallocationinfo` tool and using the `bash` shell and its tools
### By Giuseppe Amatulli

We are going to use gdallocationinfo to extract information in each raster predictors. 


    Usage: gdallocationinfo [--help-general] [-xml] [-lifonly] [-valonly]
                        [-b band]* [-overview overview_level]
                        [-l_srs srs_def] [-geoloc] [-wgs84]
                        [-oo NAME=VALUE]* srcfile x y


In [18]:
%%bash

time (
for file in $(awk '{if (NR==1) print $1="", $2="", $3="", $0}' tree_height/txt/eu_x_y_predictors_select.txt);  do 
echo $file > tree_height/txt/$file.txt
gdallocationinfo -valonly -geoloc -wgs84 tree_height/geodata_raster/$file.tif < tree_height/txt/eu_x_y_select.txt >> tree_height/txt/$file.txt
done 

paste -d " " <(awk '{ print $1,$2,$3}' tree_height/txt/eu_x_y_predictors_select.txt) \
$(for file in  $(awk '{if (NR==1)  print $1="", $2="", $3="", $0}' tree_height/txt/eu_x_y_predictors_select.txt); do ls tree_height/txt/$file.txt; done) \
> tree_height/txt/eu_x_y_predictors_select_gdal.txt 
)


real	1m50.404s
user	1m24.222s
sys	0m12.117s


Running time 

    real	1m50.404s
    user	1m24.222s
    sys 	0m12.117s

In [19]:
! head -3 tree_height/txt/eu_x_y_predictors_select_gdal.txt

ID X Y BLDFIE_WeigAver CECSOL_WeigAver CHELSA_bio18 CHELSA_bio4 convergence cti dev-magnitude eastness elev forestheight glad_ard_SVVI_max glad_ard_SVVI_med glad_ard_SVVI_min northness ORCDRC_WeigAver outlet_dist_dw_basin SBIO3_Isothermality_5_15cm SBIO4_Temperature_Seasonality_5_15cm treecover
1 6.050001 49.727499 1540 13 2113 5893 -10.4865598678589 -238043120 1.15841722488403 0.0690935924649239 353.983123779297 23 276.87109375 46.444091796875 347.665405273438 0.0424997806549072 9 780403 19.7989921569824 440.672210693359 85
2 6.0500017 49.922155 1491 12 1993 5912 33.2743606567383 -208915344 -1.75534081459045 0.269112348556519 267.511688232422 19 -49.5263671875 19.552734375 -130.541748046875 0.182779803872108 16 772777 20.8894119262695 457.756195068359 85


Of course, the same result can be achieved with direct read/write via rasterio packagein one full Python script. 

## Using Python with rasterio
### Script construction, step by step. By Francesco Lovergine

Due to the size of the dataset it is mandatory increasing the memory available for the VM and possibly add on demand swap space (i.e. virtual memory on disk), for instance via:

    sudo apt install swapspace
    
Of course, consider that eventually it could require a lot of temporary storage taken under `/var/lib/swapspace` which could be freed after use.

In [51]:
import csv
import glob
import rasterio
import os.path

In [32]:
files = []
for filename in glob.glob('tree_height/geodata_raster/[!l]*.tif'):
    ds = rasterio.open(filename, mode='r')
    files.append(ds)

In [33]:
files

[<open DatasetReader name='tree_height/geodata_raster/BLDFIE_WeigAver.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/CECSOL_WeigAver.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/CHELSA_bio18.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/CHELSA_bio4.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/ORCDRC_WeigAver.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/SBIO3_Isothermality_5_15cm.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/SBIO4_Temperature_Seasonality_5_15cm.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/convergence.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/cti.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/dev-magnitude.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/eastness.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/

In [34]:
print(len(files))

19


In [35]:
os.path.basename(files[0].name)

'BLDFIE_WeigAver.tif'

In [36]:
(name, ext) = os.path.splitext(os.path.basename(files[0].name))
name

'BLDFIE_WeigAver'

In [37]:
header = ''
for ds in files:
    field_name =   os.path.splitext(os.path.basename(ds.name))[0]
    header += ' '+field_name
header.lstrip()

'BLDFIE_WeigAver CECSOL_WeigAver CHELSA_bio18 CHELSA_bio4 ORCDRC_WeigAver SBIO3_Isothermality_5_15cm SBIO4_Temperature_Seasonality_5_15cm convergence cti dev-magnitude eastness elev forestheight glad_ard_SVVI_max glad_ard_SVVI_med glad_ard_SVVI_min northness outlet_dist_dw_basin treecover'

Starting from those files now open in rasterio, it is possible to read the band and store them in a list

In [40]:
band = files[0].read(1)
band

array([[ 1523,  1523, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       ...,
       [ 1514,  1514,  1514, ...,  1455,  1455,  1455],
       [65535, 65535, 65535, ...,  1457,  1457,  1457],
       [65535, 65535, 65535, ...,  1458,  1458,  1458]], dtype=uint16)

In [41]:
bands = []
for ds in files:
    bands.append(ds.read(1))

In [42]:
bands

[array([[ 1523,  1523, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        ...,
        [ 1514,  1514,  1514, ...,  1455,  1455,  1455],
        [65535, 65535, 65535, ...,  1457,  1457,  1457],
        [65535, 65535, 65535, ...,  1458,  1458,  1458]], dtype=uint16),
 array([[   11,    11, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        ...,
        [   16,    16,    16, ...,    20,    20,    20],
        [65535, 65535, 65535, ...,    19,    19,    19],
        [65535, 65535, 65535, ...,    19,    19,    19]], dtype=uint16),
 array([[ 2024,  2024, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        ...,
        [ 2401,  2401,  2401, ...,  4405,  4405,  4405],
        [65535, 6

Now let's have a trial for concatenating raster values

In [72]:
with open('tree_height/txt/eu_x_y_select.txt') as csvfile:
    coords = csv.reader(csvfile, delimiter=' ')
    i = 1
    for (long, lat) in coords:
        print('{} {} {} '.format(i, long, lat),end='')
        for j, ds in enumerate(files):
            idx = ds.index(float(long), float(lat))
            band = bands[j]
            val = band[idx]
            print('{} '.format(val), end='')
        print("")b
        i+=1
        if i > 10: break # just for the very first rows and check


1 6.050001 49.727499 1540 13 2113 5893 9 19.798992156982422 440.6722106933594 -10.486559867858887 -238043120 1.1584172248840332 0.06909359246492386 353.9831237792969 23 276.87109375 46.444091796875 347.6654052734375 0.04249978065490723 780403 85 
2 6.0500017 49.922155 1491 12 1993 5912 16 20.88941192626953 457.7561950683594 33.27436065673828 -208915344 -1.755340814590454 0.26911234855651855 267.5116882324219 19 -49.5263671875 19.552734375 -130.541748046875 0.18277980387210846 772777 85 
3 6.0500021 48.602377 1521 17 2124 5983 14 20.695877075195312 481.87969970703125 0.045293472707271576 -137479792 1.9087798595428467 -0.01605454459786415 389.75115966796875 21 93.25732421875 50.74365234375 384.5224609375 0.0362534299492836 898820 62 
4 6.0500089 48.151979 1526 16 2569 6130 15 19.375 479.4102783203125 -33.654273986816406 -267223072 0.9657867550849915 0.06776714324951172 380.20770263671875 27 542.4013671875 202.26416015625 386.15673828125 0.0051393029280006886 831824 85 
5 6.0500102 49.588

The first 10 rows seem correct in respect with the previous file, now it is only need to dump the rows to a file, with an appropriate header.

In [83]:
%%timeit -r 1 -n 1
with open('tree_height/txt/eu_x_y_predictors_select_new.txt', 'w') as out:
    print('ID X Y' + header, file=out)
    with open('tree_height/txt/eu_x_y_select.txt') as csvfile:
        coords = csv.reader(csvfile, delimiter=' ')
        i = 1
        for (long, lat) in coords:
            print('{} {} {} '.format(i, long, lat),end='', file=out)
            for j, ds in enumerate(files):
                idx = ds.index(float(long), float(lat))
                band = bands[j]
                val = band[idx]
                print('{} '.format(val), end='', file=out)
            print("", file=out)
            i+=1
            if i>100000: break # that's to get a fast partial result...

2min 10s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


You can notice that the output is extremely slow. Efficiency can be eventually increased by changing the buffering size from the default one.

In [86]:
! head -3 tree_height/txt/eu_x_y_predictors_select_new.txt

ID X Y BLDFIE_WeigAver CECSOL_WeigAver CHELSA_bio18 CHELSA_bio4 ORCDRC_WeigAver SBIO3_Isothermality_5_15cm SBIO4_Temperature_Seasonality_5_15cm convergence cti dev-magnitude eastness elev forestheight glad_ard_SVVI_max glad_ard_SVVI_med glad_ard_SVVI_min northness outlet_dist_dw_basin treecover
1 6.050001 49.727499 1540 13 2113 5893 9 19.798992156982422 440.6722106933594 -10.486559867858887 -238043120 1.1584172248840332 0.06909359246492386 353.9831237792969 23 276.87109375 46.444091796875 347.6654052734375 0.04249978065490723 780403 85 
2 6.0500017 49.922155 1491 12 1993 5912 16 20.88941192626953 457.7561950683594 33.27436065673828 -208915344 -1.755340814590454 0.26911234855651855 267.5116882324219 19 -49.5263671875 19.552734375 -130.541748046875 0.18277980387210846 772777 85 


### Stend-alone Python script. By Francesco Lovergine

Running the script for the whole content of 1.2M of records take quite a full time, you can run it yourself or use the resulting file stored in the staging area of the VM and clone via git.

    #!/usr/bin/env python3
    #
    # This is the whole script written in a proper way to work even in bash.
    # You can copy&paste this block in a text `whatever.py` file, then
    # chmod a+x whatever.py 
    # and run it as ./whatever.py
    #
    
    import csv
    import glob
    import rasterio
    import os.path

    files = []
    for filename in glob.glob('tree_height/geodata_raster/[!l]*.tif'):
        ds = rasterio.open(filename, mode='r')
        files.append(ds)

    (name, ext) = os.path.splitext(os.path.basename(files[0].name))
    header = ''
    for ds in files:
        field_name = os.path.splitext(os.path.basename(ds.name))[0]
        header += ' '+field_name

    print('Reading raster files...')

    bands = []
    for ds in files:
        bands.append(ds.read(1))

    print('Writing samples')

    with open('tree_height/txt/eu_x_y_predictors_select_new.txt', 'w') as out:
        print('ID X Y' + header, file=out)
        with open('tree_height/txt/eu_x_y_select.txt') as csvfile:
            coords = csv.reader(csvfile, delimiter=' ')
            i = 1
            for (long, lat) in coords:
                print('{} {} {} '.format(i, long, lat),end='', file=out)
                for j, ds in enumerate(files):
                    idx = ds.index(float(long), float(lat))
                    band = bands[j]
                    val = band[idx]
                    print('{} '.format(val), end='', file=out)
                print("", file=out)
                if not i % 10: print('Record {} ...'.format(i))
                i+=1
        csvfile.close()
    out.close()

    print('Finished')

    exit(0)

This script can be run by 

In [ ]:
! time python3 /media/sf_LVM_shared/my_SE_data/exercise/Tree_Height_02Predictors_extraction_python1.py

## Using Python with rasterio and numpy
### Script construction, step by step. By Hannah Weiser

Imports

In [26]:
from pathlib import Path
import rasterio as rio
import numpy as np

Create a list of (opened) raster datasets and of the corresponding field names

In [27]:
datasets = []
fieldnames = []
for filename in Path('tree_height/geodata_raster/').glob('[!l]*.tif'):
    ds = rio.open(filename, mode='r')
    datasets.append(ds)
    fieldnames.append(filename.stem)

In [28]:
fieldnames

['northness',
 'forestheight',
 'elev',
 'CHELSA_bio4',
 'SBIO3_Isothermality_5_15cm',
 'SBIO4_Temperature_Seasonality_5_15cm',
 'dev-magnitude',
 'BLDFIE_WeigAver',
 'CHELSA_bio18',
 'glad_ard_SVVI_min',
 'eastness',
 'ORCDRC_WeigAver',
 'cti',
 'treecover',
 'outlet_dist_dw_basin',
 'glad_ard_SVVI_med',
 'convergence',
 'glad_ard_SVVI_max',
 'CECSOL_WeigAver']

In [29]:
datasets

[<open DatasetReader name='tree_height/geodata_raster/northness.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/forestheight.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/elev.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/CHELSA_bio4.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/SBIO3_Isothermality_5_15cm.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/SBIO4_Temperature_Seasonality_5_15cm.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/dev-magnitude.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/BLDFIE_WeigAver.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/CHELSA_bio18.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/glad_ard_SVVI_min.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/eastness.tif' mode='r'>,
 <open DatasetReader name='tree_height/geodata_raster/OR

Read the coordinates to a numpy array

In [30]:
coords = np.genfromtxt('tree_height/txt/eu_x_y_select.txt', delimiter=' ')

In [31]:
coords

array([[ 6.050001 , 49.727499 ],
       [ 6.0500017, 49.922155 ],
       [ 6.0500021, 48.602377 ],
       ...,
       [ 9.9499985, 49.227932 ],
       [ 9.9499988, 49.936763 ],
       [ 9.9499996, 49.48964  ]])

In [32]:
coords.shape

(1267239, 2)

Create a dictionary with an array of pixel values at the coordinates for each field (i.e. values sampled from each raster)

In [33]:
raster_val_dict = {}
for i, ds in enumerate(datasets):
    # transform coordinates to raster row and column indices
    rows, cols = rio.transform.rowcol(ds.transform, coords[:, 0], coords[:, 1])
    rowcols = np.array(list(zip(rows, cols)))
    # read band of raster
    band = ds.read(1)
    # get values by indexing band with row and column indices
    vals = np.array([band[row, col] for row, col in rowcols])
    # add value array to dictionary
    raster_val_dict[fieldnames[i]] = vals
    ds.close()

In [34]:
raster_val_dict

{'northness': array([ 0.04249978,  0.1827798 ,  0.03625343, ..., -0.06422238,
        -0.00754081,  0.10954276], dtype=float32),
 'forestheight': array([23, 19, 21, ..., 23, 27, 23], dtype=uint8),
 'elev': array([353.98312, 267.5117 , 389.75116, ..., 422.33276, 306.65903,
        349.73883], dtype=float32),
 'CHELSA_bio4': array([5893, 5912, 5983, ..., 6622, 6495, 6588], dtype=uint16),
 'SBIO3_Isothermality_5_15cm': array([19.798992, 20.889412, 20.695877, ..., 18.91893 , 17.010637,
        17.769867], dtype=float32),
 'SBIO4_Temperature_Seasonality_5_15cm': array([440.6722 , 457.7562 , 481.8797 , ..., 490.463  , 457.73627,
        489.45746], dtype=float32),
 'dev-magnitude': array([ 1.1584172 , -1.7553408 ,  1.9087799 , ...,  0.28438434,
        -1.0900238 ,  0.52376586], dtype=float32),
 'BLDFIE_WeigAver': array([1540, 1491, 1521, ..., 1523, 1533, 1517], dtype=uint16),
 'CHELSA_bio18': array([2113, 1993, 2124, ..., 2283, 2103, 1926], dtype=uint16),
 'glad_ard_SVVI_min': array([ 347.6

Create the header of the output file

In [35]:
# get keys from dictionary
keys = raster_val_dict.keys()
keys

dict_keys(['northness', 'forestheight', 'elev', 'CHELSA_bio4', 'SBIO3_Isothermality_5_15cm', 'SBIO4_Temperature_Seasonality_5_15cm', 'dev-magnitude', 'BLDFIE_WeigAver', 'CHELSA_bio18', 'glad_ard_SVVI_min', 'eastness', 'ORCDRC_WeigAver', 'cti', 'treecover', 'outlet_dist_dw_basin', 'glad_ard_SVVI_med', 'convergence', 'glad_ard_SVVI_max', 'CECSOL_WeigAver'])

In [36]:
# add keys (=field names) to string which starts with "ID, Longitude and Latitude"
header = f"ID Longitude Latitude {' '.join(keys)}"
header

'ID Longitude Latitude northness forestheight elev CHELSA_bio4 SBIO3_Isothermality_5_15cm SBIO4_Temperature_Seasonality_5_15cm dev-magnitude BLDFIE_WeigAver CHELSA_bio18 glad_ard_SVVI_min eastness ORCDRC_WeigAver cti treecover outlet_dist_dw_basin glad_ard_SVVI_med convergence glad_ard_SVVI_max CECSOL_WeigAver'

Create a 2D array with the indixes, coordinates and corresponding raster values for each sampled point

In [37]:
# get all arrays from dictionary as list
all_vals = list(raster_val_dict.values())
# get index, x-coordinate and y-coordinate as list and add the value arrays to that list
all_vals = [np.arange(1, coords.shape[0]+1), coords[:, 0], coords[:, 1]] + all_vals

In [38]:
# stack all arrays to a 2D array
all_vals_arr = np.hstack([all_vals]).T
all_vals_arr.shape

(1267239, 22)

In [39]:
# have a look at the first three rows
all_vals_arr[:3, :]

array([[ 1.00000000e+00,  6.05000100e+00,  4.97274990e+01,
         4.24997807e-02,  2.30000000e+01,  3.53983124e+02,
         5.89300000e+03,  1.97989922e+01,  4.40672211e+02,
         1.15841722e+00,  1.54000000e+03,  2.11300000e+03,
         3.47665405e+02,  6.90935925e-02,  9.00000000e+00,
        -2.38043120e+08,  8.50000000e+01,  7.80403000e+05,
         4.64440918e+01, -1.04865599e+01,  2.76871094e+02,
         1.30000000e+01],
       [ 2.00000000e+00,  6.05000170e+00,  4.99221550e+01,
         1.82779804e-01,  1.90000000e+01,  2.67511688e+02,
         5.91200000e+03,  2.08894119e+01,  4.57756195e+02,
        -1.75534081e+00,  1.49100000e+03,  1.99300000e+03,
        -1.30541748e+02,  2.69112349e-01,  1.60000000e+01,
        -2.08915344e+08,  8.50000000e+01,  7.72777000e+05,
         1.95527344e+01,  3.32743607e+01, -4.95263672e+01,
         1.20000000e+01],
       [ 3.00000000e+00,  6.05000210e+00,  4.86023770e+01,
         3.62534299e-02,  2.10000000e+01,  3.89751160e+02,
    

Write the array to a txt-file

In [42]:
# define the format of each column
fmt = '%u %.6f %.6f %u %u %u %u %.6f %i %.8f %.8f %.5f %u %.6f %.6f %.6f %.8f %u %u %.6f %.4f %i'
# save
np.savetxt('tree_height/txt/eu_x_y_predictors_select_python2.txt', all_vals_arr, fmt=fmt, delimiter=' ', header=header, comments='')

In [41]:
# depending on how we process the data, we could also use np.save() to save the array to a binary format

### Stend-alone Python script. By Hannah Weiser


    #!/usr/bin/env python
    # coding: utf-8

    """
    Modification of the RasterIO Final Script for Preparing the Dataset for the Next ML Exercise 

    Hannah Weiser, 2023-04-22
    """

    # Imports
    from pathlib import Path
    import rasterio as rio
    import numpy as np
    import time

    start = time.time()
    print("Start processing ...")

    # Create a list of (opened) raster datasets and of the corresponding field names
    datasets = []
    fieldnames = []
    for filename in Path('tree_height/geodata_raster/').glob('[!l]*.tif'):
        ds = rio.open(filename, mode='r')
        datasets.append(ds)
        fieldnames.append(filename.stem)

    # Read the coordinates to a numpy array
    coords = np.genfromtxt('tree_height/txt/eu_x_y_select.txt', delimiter=' ')

    # Create a dictionary with an array of pixel values at the coordinates for each field (i.e. values sampled from each raster)
    raster_val_dict = {}
    for i, ds in enumerate(datasets):
        # transform coordinates to raster row and column indices
        rows, cols = rio.transform.rowcol(ds.transform, coords[:, 0], coords[:, 1])
        rowcols = np.array(list(zip(rows, cols)))
        # read band of raster
        band = ds.read(1)
        # get values by indexing band with row and column indices
        vals = np.array([band[row, col] for row, col in rowcols])
        # add value array to dictionary
        raster_val_dict[fieldnames[i]] = vals
        ds.close()

    # get keys from dictionary
    keys = raster_val_dict.keys()
    # add keys (=field names) to string which starts with "ID, Longitude and Latitude"
    header = f"ID X Y {' '.join(keys)}"


    # Create a 2D array with the indixes, coordinates and corresponding raster values for each sampled point

    # get all arrays from dictionary as list
    all_vals = list(raster_val_dict.values())
    # get index, x-coordinate and y-coordinate as list and add the value arrays to that list
    all_vals = [np.arange(1, coords.shape[0]+1), coords[:, 0], coords[:, 1]] + all_vals

    # stack all arrays to a 2D array
    all_vals_arr = np.hstack([all_vals]).T

    end_operation = time.time()
    print(f"Done in {end_operation-start:.1f} sec., Writing output file...")

    # Write the array to a txt-file

    # define the format of each column
    fmt = '%u %.6f %.6f %u %u %u %u %.6f %i %.8f %.8f %.5f %u %.6f %.6f %.6f %.8f %u %u %.6f %.4f %i'
    # save
    np.savetxt('tree_height/txt/eu_x_y_predictors_select_python2.txt', all_vals_arr, fmt=fmt, delimiter=" ", header=header, comments='')

    # depending on how we process the data later on, we could also use np.save() to save the array to a binary format

    end = time.time()

    print(f"Done. Writing took {end-end_operation:.1f} sec. Full script took {end-start:.1f}")


In [44]:
! time python3 /media/sf_LVM_shared/my_SE_data/exercise/Tree_Height_02Predictors_extraction_python2.py

Start processing ...
Done in 101.4 sec., Writing output file...
Done. Writing took 9.7 sec. Full script took 111.2

real	1m51.826s
user	1m21.891s
sys	0m14.028s


    real	1m51.826s
    user	1m21.891s
    sys	0m14.028s